# Cat vs. Dog Classification Using CNN

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

### Part 1: Data Preprocessing

#### Preprocessing Train Data

In [2]:
# using the image data generator api from keras - this contains some preprocessing formats and we can use that to perform required image preprocessing.

image_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip = True
)

training_set = image_datagen.flow_from_directory(
    'C:/Users/Bidee/OneDrive/Desktop/Data Science/Deep_Learning_A_Z/Volume 1 - Supervised Deep Learning/Part 2 - Convolutional Neural Networks (CNN)/Section 40 - Convolutional Neural Networks (CNN)/dataset/training_set',
    target_size = (150,150),
    batch_size= 32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


#### Preprocessing Test Data

In [3]:
# we dont need to apply any transformations into our test datasets as those are like new images

test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory(
    'C:/Users/Bidee/OneDrive/Desktop/Data Science/Deep_Learning_A_Z/Volume 1 - Supervised Deep Learning/Part 2 - Convolutional Neural Networks (CNN)/Section 40 - Convolutional Neural Networks (CNN)/dataset/test_set',
    target_size = (150,150),
    batch_size= 32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


### Part 2: Build The CNN

#### Initializing the CNN

In [4]:
cnn = tf.keras.models.Sequential()

#### Step 1: Convolution

In [5]:
# adding the 1st layer of convolution, here we are adding all the matrices i.e., filters, kernels etc. input shape is required for the 1st input layers of CNN.

cnn.add(tf.keras.layers.Conv2D(filters = 32,kernel_size = 2, activation = 'leaky_relu', input_shape=[150,150,3]))

#### Step 2: Pooling

In [6]:
# we are taking a max pooling size with a 2x2 matrix and stride will be 2. 

cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

#### Adding a second convolution layer

In [7]:
# while adding subsequent layers, thus forming deep neural network, we don't need input shapes anymore 

cnn.add(tf.keras.layers.Conv2D(filters = 32,kernel_size = 2, activation = 'leaky_relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

#### Step 3: Flatteing

In [8]:
# we are flattening all the convolutions into a 1D array which will be fed to the fully connected neural network.

cnn.add(tf.keras.layers.Flatten())

#### Step 4: Full Connection

In [9]:
# our good ol ANN. The only thing changed is the number of hidden layers. Since dealing with CNN is complicated its better to have a deep ANN for better learning.

cnn.add(tf.keras.layers.Dense(units = 128, activation = 'leaky_relu' ))

#### Step 5: Output layer

In [10]:
# Output layer will have 1 output and sigmoid activation as usual.
# note: If we're doing multi class classification, we would be using softmax activation function

cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid' ))

### Part 3: Training the CNN

#### Compiling the CNN

In [11]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#### Training the CNN on Training set and testing it on Test Set

In [12]:
# unlike normal training and testings, here we're training and testing it on the validation data at the same time.
with tf.device('/GPU:0'):
    cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 54s 211ms/step - loss: 0.6911 - accuracy: 0.6108 - val_loss: 0.6260 - val_accuracy: 0.6695
Epoch 2/25
250/250 [==============================] - 45s 181ms/step - loss: 0.6058 - accuracy: 0.6626 - val_loss: 0.5892 - val_accuracy: 0.7045
Epoch 3/25
250/250 [==============================] - 46s 184ms/step - loss: 0.5548 - accuracy: 0.7124 - val_loss: 0.5604 - val_accuracy: 0.7045
Epoch 4/25
250/250 [==============================] - 46s 185ms/step - loss: 0.5257 - accuracy: 0.7350 - val_loss: 0.5298 - val_accuracy: 0.7270
Epoch 5/25
250/250 [==============================] - 47s 187ms/step - loss: 0.5054 - accuracy: 0.7538 - val_loss: 0.5238 - val_accuracy: 0.7630
Epoch 6/25
250/250 [==============================] - 48s 192ms/step - loss: 0.4837 - accuracy: 0.7665 - val_loss: 0.5038 - val_accuracy: 0.7485
Epoch 7/25
250/250 [==============================] - 48s 193ms/step - loss: 0.4667 - accuracy: 0.7781 - val_loss: 0.4974 - val_ac

### Part 4: Making Prediction

In [13]:
import numpy as np 
from keras.utils import load_img, img_to_array

img_path = 'C:/Users/Bidee/OneDrive/Desktop/Data Science/Deep_Learning_A_Z/Volume 1 - Supervised Deep Learning/Part 2 - Convolutional Neural Networks (CNN)/Section 40 - Convolutional Neural Networks (CNN)/dataset/single_prediction/cat_or_dog_1.jpg'
test_image = load_img(img_path,target_size = (150,150))
#converting the target image from jpg to a numpy array which is the desired dtype.
test_image = img_to_array(test_image)
# we are expanding the dimensions cuz previously we specified the batch size. This basically means images are expected by the model in batch format.
# so, the model will take , batch 1: 32 images, batch 2: 32 images and so on to train and test. Hence it will be expecting the same when it comes to prediction.
# taking the number of batches as a separate dimension we can say one batch has a dimension of: 32,150,150.
# so, when feeding new images for classifications, we want a fake dimension containing a value of 1 as our added dimension to 150, 150 as we want to predict 
# a single image's class, the net dimension would become 1, 150, 150.
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image/255.0)


1/1 [==============================] - 0s 186ms/step


In [14]:
# now we need to do some decodings manually for final prediction.
# we can use the following code to see what encodings for the classes has been done:
training_set.class_indices

# the following is our decodings: 
if result[0][0] >0.5 :
  prediction = 'dog'
else:
  prediction = 'cat'

print(prediction)

dog
